In [40]:
# imports
import os
import torch
import torch.nn as nn
import json
import numpy as np
import pandas as pd
from PIL import Image

from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, read_file

In [2]:
dataset_path = ''

In [3]:
# check to see if we have good number of pictures
dir_path = r'dataset/img_resized'
# print(os.listdir(dir_path))
count = 0
for item in os.listdir(dir_path):
    # check if current path is a file
    count += 1
print('File count:', count)

File count: 59252


In [4]:
anns = json.load(open("dataset/MMHS150K_GT.json","r"))
print("Loaded anns: " + str(len(anns)))

majority_not_hate = 0
majority_hate = 0
majority_racist = 0
majority_sexist = 0
majority_homo = 0
majority_religion = 0
majority_other = 0

for k,v in anns.items():
    labels = []
    label_num = []
    # print(len(v["labels_str"]))
    for label in v["labels_str"]:
        if "Not" in label:
            label_num.append(0)
        elif "Racist" in label:
            label_num.append(1)
        elif "Sexist" in label:
            label_num.append(2)
        elif "Homo" in label:
            label_num.append(3)
        elif "Religion" in label:
            label_num.append(4)
        elif "Other" in label:
            label_num.append(5)
        else:
            print("Error with: " + label)
            label = "Error"

    if label_num.count(0) > 1:
        majority_not_hate+=1
    else:
        majority_hate+=1
        if label_num.count(1) > 1:
            majority_racist+=1
        elif label_num.count(2) > 1:
            majority_sexist+=1
        elif label_num.count(3) > 1:
            majority_homo+=1
        elif label_num.count(4) > 1:
            majority_religion+=1
        elif label_num.count(5) > 1:
            majority_other+=1

print("Total Tweets Majority Voting: Not Hate: " + str(majority_not_hate) + ", Hate: " + str(majority_hate) + ", Racist: " + str(majority_racist) + ", Sexist: " + str(majority_sexist) + ", Homophobe: " + str(majority_homo) + ", Religion: " + str(majority_religion) + ", Other: " + str(majority_other))

Loaded anns: 149823
Total Tweets Majority Voting: Not Hate: 112845, Hate: 36978, Racist: 11925, Sexist: 3495, Homophobe: 3870, Religion: 163, Other: 5811


In [5]:
image_path = os.path.join(dataset_path, 'dataset/img_resized')
text_path = os.path.join(dataset_path, 'dataset/img_txt')
GT_path = os.path.join(dataset_path, 'dataset/MMHS150K_GT.csv')

In [6]:
from preprocessing import create_csv_labels
# uncomment the first time
# create_csv_labels(os.path.join(dataset_path, 'dataset/MMHS150K_GT.json'), GT_path, text_path)

In [52]:
class MMHS_150KDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, GT_path, image_path, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.GT_path = GT_path
        self.GT_data = pd.read_csv(GT_path)
        self.idx_list = []
        # self.root_dir = root_dir
        self.image_path = image_path
        # self.text_path = text_path
        self.transform = transform

        self.len_samples = len(self.GT_data.iloc[:, 0])
        # self.refine_images()

    def __len__(self):
        return self.len_samples

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        ID = self.GT_data.iloc[idx, 0]
        
        # txt_path = os.path.join(self.text_path, str(ID) + '.json').replace("\\","/")
        img_path = os.path.join(self.image_path, str(ID) + '.jpg').replace("\\","/")
        # print(img_path)
        # print(txt_path)
        try:
          if self.transform:
            image = Image.open(img_path)
            image = self.transform(image)
          else:
            image = read_image(img_path)
        except:
           print(f"id = {ID} not found")

        # f = open(text_path, 'r')
        # data = json.load(f)
        # text = data['img_text']
        # text = f.read()

        # text = read_file(txt_path)

        text = self.GT_data.iloc[idx, 3]

        label = self.GT_data.iloc[idx, 2]

        # sample = {'ID': ID, 'text': text, 'label': label}
        sample = {'ID': ID, 'text': text, 'image': image, 'label': label}


        return sample

    def refine_images(self):
      # NOT USED ANYMORE ⚠️
      GT_path_cleared = os.path.join(dataset_path, 'dataset/MMHS150K_GT_cleared.csv')
      cmpt = 0
      with open(GT_path, 'r') as readFile:
        lines = readFile.readlines()
        for row in lines[1:]:
          idx = row[0:19]
          img_path = os.path.join(self.image_path, str(idx) + '.jpg').replace("\\","/")
          # look if the image exists, otherwise delete the idx from the csv
          if not (os.path.isfile(img_path)):
            lines.remove(row)
            cmpt = cmpt + 1

      with open(GT_path_cleared, 'w') as writeFile:
        for line in lines:
          writeFile.write(line)

      self.len_samples = len(lines)
      print(f"len_samples = {self.len_samples}")
      print(f"len_samples = {len(self.GT_data.iloc[:, 0])}")
      self.GT_data =  pd.read_csv(GT_path_cleared)
      # print(f"nbr_missing = {cmpt}")
      # print(f"len = {len(self.GT_data.iloc[:, 0])}")
      # print(f"linelen = {len(lines)}")

# example = MMHS_150KDataset(GT_path, image_path)
# example[1000]

In [53]:
dataset[18236]

RuntimeError: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]

In [43]:
### WeightedRandomSampler Implementation

from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from torchvision import transforms
import random

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to fit VilBERT input size
    transforms.ToTensor(),           # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

# Define the size of each subset (train, validation, test)
dataset = MMHS_150KDataset(GT_path, image_path, transform=transform)

In [48]:

dataset_size = len(dataset)
validation_split = 0.1
test_split = 0.5  # Proportion of validation set
batch_size = 16
random_seed = 42


# Get indices of all samples
indices = list(range(dataset_size))
# Shuffle the indices
np.random.seed(random_seed)
np.random.shuffle(indices)

# Split indices for validation and test sets
val_split = int(np.floor(validation_split * dataset_size))
test_split = int(np.floor(test_split * val_split))
val_indices = indices[:val_split]
test_indices = indices[val_split:(val_split + test_split)]
train_indices = indices[(val_split + test_split):]
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# only one for loop
not_hate_indices = []
hate_indices = []
for idx in train_indices:
    try:
        if dataset[idx]['label'] == 1:
            hate_indices.append(idx)
        else:
            not_hate_indices.append(idx)
    except:
        print(f"Error with idx: {idx}")
        print(f"Label: {dataset[idx]['label']}")
# not_hate_indices = [idx for idx in train_indices if dataset[idx]['label'] == 0]
# hate_indices = [idx for idx in train_indices if dataset[idx]['label'] == 1]

num_not_hate = len(not_hate_indices)
num_hate = len(hate_indices)
total_samples = num_not_hate + num_hate

# Create a WeightedRandomSampler to balance the training dataset
class_weights = [num_hate/total_samples, num_not_hate/ total_samples]  # Inverse of number of samples per class

weights = []
for idx in train_indices:
    try:
        label = dataset[idx]['label']
        according_weights = class_weights[int(label)]
        weights.append(according_weights)
    except:
        print(f"Error with idx: {idx}")
        print(f"Label: {dataset[idx]['label']}")

# weights = [class_weights[int(dataset[idx]['label'])] for idx in train_indices]
sampler = WeightedRandomSampler(weights, len(weights))

# Create data loader for balanced training set
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=sampler)

# Create data loaders for validation and test sets
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)


Error with idx: 18236


RuntimeError: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]

In [ ]:



# Get indices of all samples
indices = list(range(dataset_size))
# Shuffle the indices
np.random.seed(random_seed)
np.random.shuffle(indices)

# Split indices for validation and test sets
val_split = int(np.floor(validation_split * dataset_size))
test_split = int(np.floor(test_split * val_split))
val_indices = indices[:val_split]
test_indices = indices[val_split:(val_split + test_split)]
train_indices = indices[(val_split + test_split):]
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# only one for loop
not_hate_indices = []
hate_indices = []
for idx in train_indices:
    if dataset[idx]['label'] == 1:
        hate_indices.append(idx)
    else:
        not_hate_indices.append(idx)
# not_hate_indices = [idx for idx in train_indices if dataset[idx]['label'] == 0]
# hate_indices = [idx for idx in train_indices if dataset[idx]['label'] == 1]

num_not_hate = len(not_hate_indices)
num_hate = len(hate_indices)
total_samples = num_not_hate + num_hate

# Create a WeightedRandomSampler to balance the training dataset
class_weights = [num_hate/total_samples, num_not_hate/ total_samples]  # Inverse of number of samples per class

weights = []
for idx in train_indices:
    try:
        label = dataset[idx]['label']
        according_weights = class_weights[int(label)]
        weights.append(according_weights)
    except:
        print(f"Error with idx: {idx}")
        print(f"Label: {dataset[idx]['label']}")

# weights = [class_weights[int(dataset[idx]['label'])] for idx in train_indices]
sampler = WeightedRandomSampler(weights, len(weights))

# Create data loader for balanced training set
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=sampler)

# Create data loaders for validation and test sets
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)


In [9]:
print(type(weights))
print(len(weights))
print(len(train_indices))

<class 'list'>
50359
50359


In [12]:
# # Assumption: *get_visual_embeddings(image)* gets the visual embeddings of the image in the batch.
# from transformers import AutoTokenizer, VisualBertForPreTraining

# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
# model = VisualBertForPreTraining.from_pretrained("uclanlp/visualbert-vqa-coco-pre")


In [13]:

# inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
# visual_embeds = get_visual_embeddings(image).unsqueeze(0)
# visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
# visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)

# inputs.update(
#     {
#         "visual_embeds": visual_embeds,
#         "visual_token_type_ids": visual_token_type_ids,
#         "visual_attention_mask": visual_attention_mask,
#     }
# )
# max_length = inputs["input_ids"].shape[-1] + visual_embeds.shape[-2]
# labels = tokenizer(
#     "The capital of France is Paris.", return_tensors="pt", padding="max_length", max_length=max_length
# )["input_ids"]
# sentence_image_labels = torch.tensor(1).unsqueeze(0)  # Batch_size


# outputs = model(**inputs, labels=labels, sentence_image_labels=sentence_image_labels)
# loss = outputs.loss
# prediction_logits = outputs.prediction_logits
# seq_relationship_logits = outputs.seq_relationship_logits

In [28]:
from transformers import AutoTokenizer, VisualBertForPreTraining

# from PIL import Image

# # Define transformations for image resizing and normalization


# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Step 3: Model Setup
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = VisualBertForPreTraining.from_pretrained("uclanlp/visualbert-vqa-coco-pre")


c:\Users\adrie\.conda\envs\DL_mp\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:

# Fine-tuning parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 3

In [27]:

# Step 4: Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    
    for batch in train_loader:
        images = batch['image']
        texts = batch['text']
        labels = batch['label']

        images = images.to(device)
        texts = list(texts)
    #     images = images.to(device)
    #     texts = list(texts)  # Ensure texts are in list format
    #     inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    #     inputs.to(device)
    #     labels = labels.to(device)

    #     # outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], 
    #     #                 visual=input_images, labels=labels)
    #     outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], 
    #                     labels=labels)
    #     loss = outputs.loss
    #     total_loss += loss.item()

    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")


RuntimeError: stack expects each tensor to be equal size, but got [3, 500, 905] at entry 0 and [3, 500, 601] at entry 1